## BUSINESS PROBLEM

One of the most significant challenges in e-commerce is the accurate calculation of post-purchase ratings for products.

Solving this problem means providing more customer satisfaction for e-commerce websites, better product visibility for sellers, and a seamless shopping experience for buyers.

Another challenge is the proper sorting of product reviews.

The prominence of misleading reviews can directly impact product sales, leading to both financial loss and customer attrition.

The resolution of these two fundamental issues will result in increased sales for e-commerce platforms and sellers, as well as a smooth shopping journey for customers.




## ABOUT DATASET

Content: This dataset containing Amazon product data includes product categories along with various metadata.

Context: The product with the most reviews in the Electronics category has user ratings and reviews.

COLUMNS

*  reviewerID: User ID
*  asin: Product ID
*  reviewerName: Username
*  helpful: Helpful review rating
*  reviewText: Review
*  overall: Product rating
*  summary: Evaluation summary
*  unixReviewTime: Review time
*  reviewTime: Review time Raw
*  day_diff: Number of days since evaluation
*  helpful_yes: Number of times the review was found helpful
*  total_vote: Number of votes cast on the review

## TASK

1. In the shared dataset, users have provided ratings and reviews for a product. The aim is to evaluate the given ratings by weighting them according to the date and comparing the "initial average rating" with the "weighted rating based on the date" to be obtained.

2. Then,the reviews will be sorted.

## PREPARING DATA



In [1]:
import pandas as pd
import math
import scipy.stats as st
from sklearn.preprocessing import MinMaxScaler

In [2]:
pd.set_option("display.max_columns",None)
pd.set_option("display.max_rows",None)
pd.set_option("display.width",500)
pd.set_option("display.expand_frame_repr",False)
pd.set_option("display.float_format",lambda x: "%.5f" % x)

In [3]:
df=pd.read_csv("/kaggle/input/amazon-reviews/amazon_reviews.csv")

In [4]:
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,day_diff,helpful_yes,total_vote
0,A3SBTW3WS4IQSN,B007WTAJTO,NaN,"[0, 0]",No issues.,4.00000,Four Stars,1406073600,2014-07-23,138,0,0
1,A18K1ODH1I2MVB,B007WTAJTO,0mie,"[0, 0]","Purchased this for my device, it worked as adv...",5.00000,MOAR SPACE!!!,1382659200,2013-10-25,409,0,0
2,A2FII3I2MBMUIA,B007WTAJTO,1K3,"[0, 0]",it works as expected. I should have sprung for...,4.00000,nothing to really say....,1356220800,2012-12-23,715,0,0
3,A3H99DFEG68SR,B007WTAJTO,1m2,"[0, 0]",This think has worked out great.Had a diff. br...,5.00000,Great buy at this price!!! *** UPDATE,1384992000,2013-11-21,382,0,0
4,A375ZM4U047O79,B007WTAJTO,2&amp;1/2Men,"[0, 0]","Bought it with Retail Packaging, arrived legit...",5.00000,best deal around,1373673600,2013-07-13,513,0,0


In [5]:
df.shape

(4915, 12)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4915 entries, 0 to 4914
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   reviewerID      4915 non-null   object 
 1   asin            4915 non-null   object 
 2   reviewerName    4914 non-null   object 
 3   helpful         4915 non-null   object 
 4   reviewText      4914 non-null   object 
 5   overall         4915 non-null   float64
 6   summary         4915 non-null   object 
 7   unixReviewTime  4915 non-null   int64  
 8   reviewTime      4915 non-null   object 
 9   day_diff        4915 non-null   int64  
 10  helpful_yes     4915 non-null   int64  
 11  total_vote      4915 non-null   int64  
dtypes: float64(1), int64(4), object(7)
memory usage: 460.9+ KB


In [7]:
df["overall"].value_counts()

overall
5.00000    3922
4.00000     527
1.00000     244
3.00000     142
2.00000      80
Name: count, dtype: int64

## INITIAL AVERAGE RATING

In [8]:
df.overall.mean() ##initial average rating(4.587589013224822)

4.587589013224822

## TIME-BASED WEIGHTED AVERAGE

In [9]:
df.day_diff.head()

0    138
1    409
2    715
3    382
4    513
Name: day_diff, dtype: int64

In [10]:
df.day_diff.value_counts().head()

day_diff
343    26
160    23
129    22
310    21
452    20
Name: count, dtype: int64

In [11]:
df.day_diff.describe().T

count   4915.00000
mean     437.36704
std      209.43987
min        1.00000
25%      281.00000
50%      431.00000
75%      601.00000
max     1064.00000
Name: day_diff, dtype: float64

In [12]:
df["day_diff_scaled"]=MinMaxScaler(feature_range=(1,5)).\
    fit(df[["day_diff"]]).\
    transform(df[["day_diff"]]) 

###The range of the 'day_diff' column has been created between 1 and 5. 
#And has been created a new column.

In [13]:
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,day_diff,helpful_yes,total_vote,day_diff_scaled
0,A3SBTW3WS4IQSN,B007WTAJTO,NaN,"[0, 0]",No issues.,4.00000,Four Stars,1406073600,2014-07-23,138,0,0,1.51552
1,A18K1ODH1I2MVB,B007WTAJTO,0mie,"[0, 0]","Purchased this for my device, it worked as adv...",5.00000,MOAR SPACE!!!,1382659200,2013-10-25,409,0,0,2.53528
2,A2FII3I2MBMUIA,B007WTAJTO,1K3,"[0, 0]",it works as expected. I should have sprung for...,4.00000,nothing to really say....,1356220800,2012-12-23,715,0,0,3.68674
3,A3H99DFEG68SR,B007WTAJTO,1m2,"[0, 0]",This think has worked out great.Had a diff. br...,5.00000,Great buy at this price!!! *** UPDATE,1384992000,2013-11-21,382,0,0,2.43368
4,A375ZM4U047O79,B007WTAJTO,2&amp;1/2Men,"[0, 0]","Bought it with Retail Packaging, arrived legit...",5.00000,best deal around,1373673600,2013-07-13,513,0,0,2.92662


After assigning values from 1 to 5 in ascending order based on the number of days elapsed since the evaluation, the weights in the calculation were determined as follows:

* Those with a day_diff_scaled value of 1 are assigned 28.
* Those with a day_diff_scaled value >1 and <=2 are assigned 22.
* Those with a day_diff_scaled value >2 and <=3 are assigned 18.
* Those with a day_diff_scaled value >3 and <=4 are assigned 15.
* Those with a day_diff_scaled value >4 and <5 are assigned 12.
* Those with a day_diff_scaled value of 5 are assigned 5


In [14]:
df.loc[df["day_diff_scaled"]==1,"overall"].count()

df.loc[df["day_diff_scaled"]==5,"overall"].count()

1

In [15]:
df.loc[df["day_diff_scaled"]==1,"overall"]


1494   5.00000
1688   5.00000
2603   4.00000
2629   5.00000
2712   5.00000
2793   5.00000
3690   4.00000
3735   5.00000
3741   5.00000
3742   5.00000
4364   5.00000
4507   5.00000
Name: overall, dtype: float64

In [16]:
df.loc[df["day_diff_scaled"] == 1, "overall"].mean() * 28/100 + \
    df.loc[(df["day_diff_scaled"] > 1) & (df["day_diff_scaled"] <= 2), "overall"].mean() * 22/100 + \
    df.loc[(df["day_diff_scaled"] > 2) & (df["day_diff_scaled"] <= 3), "overall"].mean() * 18/100 + \
    df.loc[(df["day_diff_scaled"] > 3) & (df["day_diff_scaled"]<=4), "overall"].mean() * 15/100 +\
    df.loc[(df["day_diff_scaled"])>4 &(df["day_diff_scaled"]<5), "overall"].mean() *12/100 +\
    df.loc[df["day_diff_scaled"]==5,"overall"].mean()*5/100


4.689651501198623

In [17]:
def time_based_weighted_average(dataframe, w1=28, w2=22, w3=18, w4=15,w5=12,w6=5):
    return dataframe.loc[df["day_diff_scaled"] == 1, "overall"].mean() * 28/100 + \
    dataframe.loc[(df["day_diff_scaled"] > 1) & (dataframe["day_diff_scaled"] <= 2), "overall"].mean() * 22/100 + \
    dataframe.loc[(df["day_diff_scaled"] > 2) & (dataframe["day_diff_scaled"] <= 3), "overall"].mean() * 18/100 + \
    dataframe.loc[(df["day_diff_scaled"] > 3) & (dataframe["day_diff_scaled"]<=4), "overall"].mean() * 15/100 +\
    dataframe.loc[(df["day_diff_scaled"])>4 &(dataframe["day_diff_scaled"]<5), "overall"].mean() *12/100 +\
    dataframe.loc[df["day_diff_scaled"]==5,"overall"].mean()*5/100

##The operation was transformed into a function

In [18]:
time_based_weighted_average(df) 

##(4.689651501198623) this figure is the time based weighted average

4.689651501198623

## SORTING REVIEWS

In [19]:
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,day_diff,helpful_yes,total_vote,day_diff_scaled
0,A3SBTW3WS4IQSN,B007WTAJTO,NaN,"[0, 0]",No issues.,4.00000,Four Stars,1406073600,2014-07-23,138,0,0,1.51552
1,A18K1ODH1I2MVB,B007WTAJTO,0mie,"[0, 0]","Purchased this for my device, it worked as adv...",5.00000,MOAR SPACE!!!,1382659200,2013-10-25,409,0,0,2.53528
2,A2FII3I2MBMUIA,B007WTAJTO,1K3,"[0, 0]",it works as expected. I should have sprung for...,4.00000,nothing to really say....,1356220800,2012-12-23,715,0,0,3.68674
3,A3H99DFEG68SR,B007WTAJTO,1m2,"[0, 0]",This think has worked out great.Had a diff. br...,5.00000,Great buy at this price!!! *** UPDATE,1384992000,2013-11-21,382,0,0,2.43368
4,A375ZM4U047O79,B007WTAJTO,2&amp;1/2Men,"[0, 0]","Bought it with Retail Packaging, arrived legit...",5.00000,best deal around,1373673600,2013-07-13,513,0,0,2.92662


In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4915 entries, 0 to 4914
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   reviewerID       4915 non-null   object 
 1   asin             4915 non-null   object 
 2   reviewerName     4914 non-null   object 
 3   helpful          4915 non-null   object 
 4   reviewText       4914 non-null   object 
 5   overall          4915 non-null   float64
 6   summary          4915 non-null   object 
 7   unixReviewTime   4915 non-null   int64  
 8   reviewTime       4915 non-null   object 
 9   day_diff         4915 non-null   int64  
 10  helpful_yes      4915 non-null   int64  
 11  total_vote       4915 non-null   int64  
 12  day_diff_scaled  4915 non-null   float64
dtypes: float64(2), int64(4), object(7)
memory usage: 499.3+ KB


In [21]:
df["helpful_no"]=df["total_vote"] - df["helpful_yes"]

In [22]:
df.head(20)

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,day_diff,helpful_yes,total_vote,day_diff_scaled,helpful_no
0,A3SBTW3WS4IQSN,B007WTAJTO,NaN,"[0, 0]",No issues.,4.00000,Four Stars,1406073600,2014-07-23,138,0,0,1.51552,0
1,A18K1ODH1I2MVB,B007WTAJTO,0mie,"[0, 0]","Purchased this for my device, it worked as adv...",5.00000,MOAR SPACE!!!,1382659200,2013-10-25,409,0,0,2.53528,0
2,A2FII3I2MBMUIA,B007WTAJTO,1K3,"[0, 0]",it works as expected. I should have sprung for...,4.00000,nothing to really say....,1356220800,2012-12-23,715,0,0,3.68674,0
3,A3H99DFEG68SR,B007WTAJTO,1m2,"[0, 0]",This think has worked out great.Had a diff. br...,5.00000,Great buy at this price!!! *** UPDATE,1384992000,2013-11-21,382,0,0,2.43368,0
4,A375ZM4U047O79,B007WTAJTO,2&amp;1/2Men,"[0, 0]","Bought it with Retail Packaging, arrived legit...",5.00000,best deal around,1373673600,2013-07-13,513,0,0,2.92662,0
5,A2IDCSC6NVONIZ,B007WTAJTO,2Cents!,"[0, 0]",It's mini storage. It doesn't do anything els...,5.00000,Not a lot to really be said,1367193600,2013-04-29,588,0,0,3.20884,0
6,A26YHXZD5UFPVQ,B007WTAJTO,2K1Toaster,"[0, 0]",I have it in my phone and it never skips a bea...,5.00000,Works well,1382140800,2013-10-19,415,0,0,2.55786,0
7,A3CW0ZLUO5X2B1,B007WTAJTO,"35-year Technology Consumer ""8-tracks to 802.11""","[0, 0]",It's hard to believe how affordable digital ha...,5.00000,32 GB for less than two sawbucks...what's not ...,1404950400,2014-10-07,62,0,0,1.22954,0
8,A2CYJO155QP33S,B007WTAJTO,4evryoung,"[1, 1]",Works in a HTC Rezound. Was running short of ...,5.00000,Loads of room,1395619200,2014-03-24,259,1,1,1.97084,0
9,A2S7XG3ZC4VGOQ,B007WTAJTO,53rdcard,"[0, 0]","in my galaxy s4, super fast card, and am total...",5.00000,works great,1381449600,2013-11-10,393,0,0,2.47507,0


"score_pos_neg_diff",
"score_average_rating",
and "Wilson Lower Bound" scores will be calculated and added to the dataset.

In [23]:
def score_pos_neg_diff(pos, neg):
    return pos-neg

In [24]:
df["score_pos_neg_diff"]=df.apply(lambda x: score_pos_neg_diff(x["helpful_yes"],x["helpful_no"]), axis=1)

In [25]:
def score_average_rating (pos,neg):
    if pos + neg == 0:
        return 0
    return pos/(pos+neg)

In [26]:
df["score_average_rating"]=df.apply(lambda x: score_average_rating(x["helpful_yes"],x["helpful_no"]), axis=1)

In [27]:
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,day_diff,helpful_yes,total_vote,day_diff_scaled,helpful_no,score_pos_neg_diff,score_average_rating
0,A3SBTW3WS4IQSN,B007WTAJTO,NaN,"[0, 0]",No issues.,4.00000,Four Stars,1406073600,2014-07-23,138,0,0,1.51552,0,0,0.00000
1,A18K1ODH1I2MVB,B007WTAJTO,0mie,"[0, 0]","Purchased this for my device, it worked as adv...",5.00000,MOAR SPACE!!!,1382659200,2013-10-25,409,0,0,2.53528,0,0,0.00000
2,A2FII3I2MBMUIA,B007WTAJTO,1K3,"[0, 0]",it works as expected. I should have sprung for...,4.00000,nothing to really say....,1356220800,2012-12-23,715,0,0,3.68674,0,0,0.00000
3,A3H99DFEG68SR,B007WTAJTO,1m2,"[0, 0]",This think has worked out great.Had a diff. br...,5.00000,Great buy at this price!!! *** UPDATE,1384992000,2013-11-21,382,0,0,2.43368,0,0,0.00000
4,A375ZM4U047O79,B007WTAJTO,2&amp;1/2Men,"[0, 0]","Bought it with Retail Packaging, arrived legit...",5.00000,best deal around,1373673600,2013-07-13,513,0,0,2.92662,0,0,0.00000


In [28]:
def wilson_lower_bound(up, down, confidence=0.95):
    n = up + down
    if n == 0:
        return 0
    z = st.norm.ppf(1 - (1 - confidence) / 2)
    phat = 1.0 * up / n
    return (phat + z * z / (2 * n) - z * math.sqrt((phat * (1 - phat) + z * z / (4 * n)) / n)) / (1 + z * z / n)

##Provides the opportunity to score any item, product or review that 
#has dual interaction. like-dislike, help-or not.


##Technique: Computes a confidence interval for the Bernoulli parameter p. 
##It accepts the lower bound of the confidence interval as the WLB score. 
##p is the probability of observing an event

In [29]:
wilson_lower_bound(800,200)

0.7740810353518655

In [30]:
df["wilson_lower_bound"]=df.apply(lambda x: wilson_lower_bound(x["helpful_yes"],x["helpful_no"]),axis=1)

In [31]:
df[["helpful_yes", "helpful_no","score_pos_neg_diff","score_average_rating","wilson_lower_bound"]].sort_values(by="wilson_lower_bound", ascending=False).head(20)

,helpful_yes,helpful_no,score_pos_neg_diff,score_average_rating,wilson_lower_bound
2031,1952,68,1884,0.96634,0.95754
3449,1428,77,1351,0.94884,0.93652
4212,1568,126,1442,0.92562,0.91214
317,422,73,349,0.85253,0.81858
4672,45,4,41,0.91837,0.80811
1835,60,8,52,0.88235,0.78465
3981,112,27,85,0.80576,0.73214
3807,22,3,19,0.88000,0.70044
4306,51,14,37,0.78462,0.67033
4596,82,27,55,0.75229,0.66359


## RESULT

In summary, when considering the "wilson_lower_bound" proves to be a more accurate measure of ranking success. For instance, when examining the reviews with indices such as 4072, 1072, 2583, or 121, it becomes evident that there are no negative comments, but the count of positive reviews is significantly low. As a result, the score_average_rating of 1 may present a biased perspective. In contrast, the wilson_lower_bound takes into consideration both helpful feedback and review frequency.